<a href="https://colab.research.google.com/github/mnylah/QNC_fall2025/blob/main/Confidence_interval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from scipy import stats

# For reproducibility
np.random.seed(42)

# Population parameters
mu = 10
sigma = 2

# Confidence level
confidence_level = 0.95
alpha = 1 - confidence_level
z_critical = stats.norm.ppf(1 - alpha/2)

# Sample sizes to evaluate
sample_sizes = [5, 10, 20, 40, 80, 160, 1000]

# For Bayesian credible interval (normal prior)
prior_mu = 0
prior_sigma2 = 1e6  # Very vague prior

# Number of bootstrap samples
n_bootstrap = 10000

# Store results
results = []

for n in sample_sizes:
    # Simulate sample from N(10, 4)
    sample = np.random.normal(loc=mu, scale=sigma, size=n)
    sample_mean = np.mean(sample)
    sample_std = np.std(sample, ddof=1)
    stderr = sample_std / np.sqrt(n)

    # 1. Z interval (σ known)
    z_margin = z_critical * (sigma / np.sqrt(n))
    z_ci = (sample_mean - z_margin, sample_mean + z_margin)

    # 2. t interval (σ unknown)
    t_critical = stats.t.ppf(1 - alpha/2, df=n-1)
    t_margin = t_critical * stderr
    t_ci = (sample_mean - t_margin, sample_mean + t_margin)

    # 3. Percentile Bootstrap CI
    bootstrap_means = np.array([
        np.mean(np.random.choice(sample, size=n, replace=True))
        for _ in range(n_bootstrap)
    ])
    bootstrap_ci = (
        np.percentile(bootstrap_means, 100 * alpha / 2),
        np.percentile(bootstrap_means, 100 * (1 - alpha / 2))
    )

    # 4. Bayesian credible interval
    posterior_variance = 1 / (n / sigma**2 + 1 / prior_sigma2)
    posterior_mean = posterior_variance * (np.sum(sample) / sigma**2 + prior_mu / prior_sigma2)
    posterior_std = np.sqrt(posterior_variance)
    bayes_ci = (
        posterior_mean - z_critical * posterior_std,
        posterior_mean + z_critical * posterior_std
    )

    # Store all results
    results.append({
        'n': n,
        'Z CI': z_ci,
        't CI': t_ci,
        'Bootstrap CI': bootstrap_ci,
        'Bayesian CI': bayes_ci
    })

# Display results
for res in results:
    print(f"Sample size: n = {res['n']}")
    print(f"  Z Interval        : {res['Z CI']}")
    print(f"  t Interval        : {res['t CI']}")
    print(f"  Bootstrap CI      : {res['Bootstrap CI']}")
    print(f"  Bayesian CI       : {res['Bayesian CI']}")
    print('-' * 60)


Sample size: n = 5
  Z Interval        : (np.float64(9.16496086749701), np.float64(12.671051029803335))
  t Interval        : (np.float64(9.15923305157548), np.float64(12.676778845724865))
  Bootstrap CI      : (np.float64(9.900751520488877), np.float64(12.031405522461434))
  Bayesian CI       : (np.float64(9.16495283431685), np.float64(12.671041594187955))
------------------------------------------------------------
Sample size: n = 10
  Z Interval        : (np.float64(9.094449361905331), np.float64(11.573629491123578))
  t Interval        : (np.float64(9.290791538589884), np.float64(11.377287314439025))
  Bootstrap CI      : (np.float64(9.5358917022701), np.float64(11.250350232638855))
  Bayesian CI       : (np.float64(9.094445476209154), np.float64(11.573625109591523))
------------------------------------------------------------
Sample size: n = 20
  Z Interval        : (np.float64(9.63572727971144), np.float64(11.388772360864605))
  t Interval        : (np.float64(9.761026538129356